In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/cosmoquester/aibookathon/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# huggingface 모델 이름이나 경로를 입력해주세요.
model_name: str = "khu-bot/polyglot-essayist-with-sum"
tokenizer_name: str = model_name
# Private으로 huggingface model hub에 올라가 있는 경우에 필요합니다.
auth_token: str = None

prompt_max_length = 128
max_length: int = 512
device: str = "cuda"

In [ ]:
tokenizer_kwargs = {"padding_side": "left"}
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_auth_token=auth_token, **tokenizer_kwargs)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=auth_token).to(device)

Downloading: 100%|██████████| 392/392 [00:00<00:00, 267kB/s]
Downloading: 100%|██████████| 1.65M/1.65M [00:01<00:00, 1.26MB/s]
Downloading: 100%|██████████| 626/626 [00:00<00:00, 157kB/s]
Downloading: 100%|██████████| 2.76G/2.76G [02:02<00:00, 22.5MB/s]   


In [ ]:
def datum_to_string(datum):
    text = f"제목: {datum['title']}\n"

    summarizations = datum.get("summarizations")
    if summarizations:
        summarization = " ".join(summarizations).replace("\n", " ")
        text = f"요약: {summarization}\n" + text
    return text

input_examples = [
    {"title": "나에게 자동차란?", "summarizations": []},
    {"title": "모기", "summarizations": []},
    {"title": "노랑 사랑", "summarizations": []},
    {"title": "깊은 호수", "summarizations": [
      "깊은 호수 같은 마음을 가진 사람이 되기를 바라는 사람은 어떤 모습인가, 깊은 호수 같은 마음을 가진 사람이 되기를 바라는 사람은 어떤 모습인가, 깊은 호수 같은 마음을 가진 사람이 되기를 바라는 사람은 어떤 모습인가, 깊은 호수 같은 마음을 가진 사람이 되기를 바라는 사람은 어떤 모습인가, 깊은 호수 같은 마음을 가진"
    ]},
    {"title": "전역 후에 헬스를 시작한 방법", "summarizations": [
      "군대에서 배운 딥스, 턱걸이 그리고 가슴 운동은 스쿼트를 기초로 시작했지만 초보자가 운동을 지속한 방법과, 과잠 입은 선후배들이 갑옷 대신 과잠 입은 선후배들이다.",
      "군대에서 이후 미디어 속 인물들의 몸짱이 비현실적이라는 생각을 버리고 운동의 빈도가 전보다 높아졌으며 운동의 빈도가 전보다 높아졌다."
    ]},
]

prompts = [datum_to_string(example) for example in input_examples]

In [ ]:
prompts

['제목: 나에게 자동차란?\n',
 '제목: 모기\n',
 '제목: 노랑 사랑\n',
 '요약: 깊은 호수 같은 마음을 가진 사람이 되기를 바라는 사람은 어떤 모습인가, 깊은 호수 같은 마음을 가진 사람이 되기를 바라는 사람은 어떤 모습인가, 깊은 호수 같은 마음을 가진 사람이 되기를 바라는 사람은 어떤 모습인가, 깊은 호수 같은 마음을 가진 사람이 되기를 바라는 사람은 어떤 모습인가, 깊은 호수 같은 마음을 가진\n제목: 깊은 호수\n',
 '요약: 군대에서 배운 딥스, 턱걸이 그리고 가슴 운동은 스쿼트를 기초로 시작했지만 초보자가 운동을 지속한 방법과, 과잠 입은 선후배들이 갑옷 대신 과잠 입은 선후배들이다. 군대에서 이후 미디어 속 인물들의 몸짱이 비현실적이라는 생각을 버리고 운동의 빈도가 전보다 높아졌으며 운동의 빈도가 전보다 높아졌다.\n제목: 전역 후에 헬스를 시작한 방법\n']

In [ ]:
input_ids = tokenizer(
        prompts,
        add_special_tokens=True,
        max_length=prompt_max_length,
        padding="longest",
        return_tensors="pt",
        return_token_type_ids=False,
        return_attention_mask=False,
)["input_ids"].to(device)

In [ ]:
# 생성하는 부분입니다. 필요에 따라 옵션을 추가하시면 됩니다.
outputs = model.generate(input_ids, 
    max_length, 
    do_sample=True,
    num_return_sequences=1,
    pad_token_id=tokenizer.pad_token_id
)

In [ ]:
texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for text in texts:
    print(text)
    print()

제목: 나에게 자동차란?
결혼 후, 나의 차가 생긴지 6개월이 다 되어간다.
1년에 한 번 이상은 꼭 나의 자동차를 점검하고, 문제가 보이면 해결을 해온다.
나의 자동차를 타는 사람들이 내 차를 운전하게 된다는 것을 알기 때문임을 인정한다.
이번에는 1일 날, 내 차를 지하 주차장에 주차하다가 주차되어 있는 내 차가 벽에 긁히는 일이 있었다.
"어? 내 차 긁히네, 뭐야 왜 이러지?"
사실 나는 이 정도의 기스가 손 쓸 틈도 없이 보이면, 그냥 눈 뜨고 잊어버리고 말았다.
"수리할 때가 됐네, 그냥 수리해. 신경 안 쓸려고 해도 자꾸 신경 쓰게 돼."
하지만, 나는 나를 바라보는 딸의 눈빛이 생각나서인지, 안 되겠다 싶었다.
"엄마, 아빠가 알아서 해!"
"근데, 내 자동차가 왜? 언제부터? 언제부터 이렇게 차가 있는데?"
"응? 엄마는 모르겠는데?"
"응? 어떻게 모르지? 엄마 친구들 차는 다 엄마 거였는데.."
"응, 엄마 친구들도 내 거를 다들 타고 다녔어. 그리고 엄마 친구가 사고 난 일이 있었거든. 나도 기억나더라. 그래서 알았어."
이번 주, 내가 타는 자동차에 작은 스크래치가 났다.
나는 자동차 보험회사에 전화를 했었다.
나: 안녕하세요? 제 차가 긁혔거든요. 저 어떻게 해야 할까요? 보험회사에서는 수리비가 얼만 지 알려주긴 했는데..저는 그 돈 받으면 별로 기분이 안 좋을 것 같아서요.. 혹시, 제가 수리를 하면 가격은 어떻게 될까요?
나: 글쎄요, 잘 모르겠는데..
나: 그럼 저도 견적을 내고 해주세요. 아님

제목: 모기
"할머니, 여기는 왜 해가 안 지지? 아 해 안 져~"
할머니 집에서 나와 횡단보도를 건너며 나는 할머니에게 묻는다. '여름엔 왜 해가 안 지냐고요.' 내가 할머니께 해가 언제냐고 묻던 마음을 나는 아직도 기억한다. 어쩌면, 할머니가 말하지 않기를 바랐던 마음으로 해를 숨겼다.
내가 말하지 않기를 바랐던 건, 무엇이든 쉽게 열어 이야기하는 편이 아니었다. 나는 언제나 누구보다 내 마음을 더 지키고 싶었다. 그래서 '모기가 너